In [20]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	#trainY = to_categorical(trainY)
	#testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [4]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [5]:
trainX, trainY, testX, testY = load_dataset()
trainX, testX = prep_pixels(trainX, testX)

In [6]:
clf1 = tf.keras.models.load_model("cifar-vgg-baseline-holdout-100epochs_all_transfer.npy")

In [7]:
print(type(clf1))

<class 'keras.engine.sequential.Sequential'>


In [8]:
clf2 = tf.keras.models.load_model("cifar-mobileNet-baseline-holdout-100epochs_all_transfer.npy")

In [9]:
clf3 = tf.keras.models.load_model("cifar-resnet50-baseline-holdout-100epochs_all_transfer.npy")

# VGG + Flatting + Gaussian

In [10]:
clf1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                        

In [11]:
vgg_lr_flatting = keras.Model(inputs=clf1.input, outputs=clf1.get_layer(index=18).output)
vgg_featureVector_flatting = vgg_lr_flatting.predict(trainX)

1563/1563 [==============================] - 8s 3ms/step


In [12]:
vgg_featureVector_flatting.shape

(50000, 2048)

In [13]:
VGG_LR_FLATTING = GaussianNB().fit(vgg_featureVector_flatting, np.ravel(trainY))

In [14]:
vgg_featureVector_flatting_test = vgg_lr_flatting.predict(testX)

313/313 [==============================] - 1s 3ms/step


In [15]:
acc = VGG_LR_FLATTING.score(vgg_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 79.700


# MobileNet + Flatting + LogisticRegression

In [16]:
clf2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 1, 1, 1024)       3228864   
 nal)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 32)          294944    
                                                                 
 batch_normalization_3 (Batc  (None, 1, 1, 32)         128       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 1, 1, 32)          0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 32)         0         
 2D)                                                             
                                                      

In [17]:
mobile_lr_flatting = keras.Model(inputs=clf2.input, outputs=clf2.get_layer(index=9).output)
mobile_featureVector_flatting = mobile_lr_flatting.predict(trainX)

1563/1563 [==============================] - 12s 7ms/step


In [18]:
mobile_featureVector_flatting.shape

(50000, 64)

In [22]:
mobile_LR_FLATTING = RandomForestClassifier(n_estimators=100, random_state=1).fit(mobile_featureVector_flatting, np.ravel(trainY))

In [23]:
mobile_featureVector_flatting_test = mobile_lr_flatting.predict(testX)

313/313 [==============================] - 2s 7ms/step


In [24]:
acc = mobile_LR_FLATTING.score(mobile_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 86.110


# Resnet + Flatting + LogisticRegression

In [25]:
clf3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 1, 1, 32)          589856    
                                                                 
 batch_normalization (BatchN  (None, 1, 1, 32)         128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1, 1, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 1, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 1, 64)          1

In [26]:
resnet_lr_flatting = keras.Model(inputs=clf3.input, outputs=clf3.get_layer(index=9).output)
resnet_featureVector_flatting = resnet_lr_flatting.predict(trainX)

1563/1563 [==============================] - 30s 18ms/step


In [27]:
resnet_featureVector_flatting.shape

(50000, 64)

In [28]:
resnet_LR_FLATTING = LogisticRegression(random_state=0).fit(resnet_featureVector_flatting, np.ravel(trainY))

In [29]:
resnet_featureVector_flatting_test = resnet_lr_flatting.predict(testX)

313/313 [==============================] - 6s 18ms/step


In [30]:
acc = resnet_LR_FLATTING.score(resnet_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 85.750


# Resnet + Pooling + LogisticRegression

In [31]:
#new_model_lr_pooling = keras.Model(inputs=clf3.input, outputs=clf3.get_layer(index=8).output)
#featureVector_pooling = new_model_lr_pooling.predict(trainX)

In [32]:
#featureVector_pooling.shape

In [33]:
#LR_POOLING = LogisticRegression(random_state=0).fit(featureVector_pooling, np.ravel(trainY))

In [34]:
#featureVector_pooling_test = new_model_lr_pooling.predict(testX)

In [35]:
#acc = LR_POOLING.score(featureVector_pooling_test, testY)
#print('Acurácia: %.3f' % (acc * 100.0))

# Emsemble Gaussian + RandomForest + LogisticRegression (VGG , MobileNet, Resnet)

In [36]:
#creating the emsemble
eclf = VotingClassifier(estimators=[('lr_vgg', VGG_LR_FLATTING), ('gaussian_mobile', mobile_LR_FLATTING), ('rf_resnet', resnet_LR_FLATTING)], voting='soft')

In [37]:
#joining the features obtained by flatting on all models
emsemble_feature_vector = np.append(vgg_featureVector_flatting, mobile_featureVector_flatting, axis=1)
emsemble_feature_vector = np.append(emsemble_feature_vector, resnet_featureVector_flatting, axis=1)

In [38]:
emsemble_feature_vector.shape

(50000, 2176)

In [39]:
#training the emsemble with all features
eclf.fit(emsemble_feature_vector, np.ravel(trainY))

VotingClassifier(estimators=[('lr_vgg', GaussianNB()),
                             ('gaussian_mobile',
                              RandomForestClassifier(random_state=1)),
                             ('rf_resnet', LogisticRegression(random_state=0))],
                 voting='soft')

In [40]:
emsemble_feature_vector_test = np.append(vgg_featureVector_flatting_test, mobile_featureVector_flatting_test, axis=1)
emsemble_feature_vector_test = np.append(emsemble_feature_vector_test, resnet_featureVector_flatting_test, axis=1)

In [42]:
acc = eclf.score(emsemble_feature_vector_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 88.590
